In [1]:
# pip install librosa

In [2]:
'''
Please install this specific version of resampy for librosa to work without errors.
'''

'\nPlease install this specific version of resampy for librosa to work without errors.\n'

In [3]:
# pip install resampy==0.3.1

In [4]:
import soundfile
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings('ignore')

In [5]:
emotions ={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

### Data for binary classification


In [6]:
def load_extract_features(data_path):

    '''
    load_extract_features() is a function that is used to load all the audio files one at a time, compute their features and return the features as well as the target values.

    There are around 8-10 audio files which are corrupted. We hardcode zero values for such files in order to maintain consistency.

    ['calm', 'happy'] emotion data is categorized into 'positive' and  ['angry', 'fearful'] into 'negative'

    Returns:
    1. Features
    2. Binary Target Values
    '''
    final_features,target_emotions, binary_label = [],[], []
    count = 0
    
    for i in glob.glob(data_path + "/Actor_*/*.wav"): #Loop to read every file.
        
        name = os.path.basename(i)
        #We split the name of the file to understand the emotion associated with the file.
        split = name.split("-")
        #We know that the third identifier is associated with the emotion of the audio file. Hence, we use [2] as it represents the third identifier.
        emotion = emotions[split[2]]

        #Below is the code to categorize the emotions into two classes to make this a binary problem.
        if emotion in ['calm', 'happy']:
            binary_label.append(0)
        elif emotion in ['angry', 'fearful']:
            binary_label.append(1)
        else:
            continue
        
        with soundfile.SoundFile(i) as audio:
            waveform = audio.read(dtype="float32")
            sr = audio.samplerate
            
            #Below is the code to extract the Mel spectrogram features
            #128 is the standard for machine learning applications using Mel spectrograms
            m_feature = librosa.feature.melspectrogram(y=waveform, sr=sr, n_mels=128, fmax=sr / 2.0).T
            melspectrogram = np.mean(m_feature,axis=0)
            if melspectrogram.shape != (128,):
                melspectrogram = np.zeros(128)
            
            #Below is the code to extract the chromagram features
            stft_wave = librosa.stft(waveform)
            stft = np.abs(stft_wave)
            c_feature = librosa.feature.chroma_stft(S=stft, sr=sr).T
            chromagram = np.mean(c_feature,axis=0)
            
            #12 is the number of pitch classes
            if chromagram.shape != (12,):
                chromagram = np.zeros(12)
                
            features=np.array([])
            features=np.hstack((chromagram, melspectrogram))
        
            final_features.append(features)
            target_emotions.append(emotion)
            
            count += 1
            if count % 100 == 0:
                print("Processed Audio File Number: ", count)
    
    #We return the features and the binary target values.
    return np.array(final_features), np.array(binary_label)

In [7]:
#Please change the path below to the path of the folder saved in your computer.
# data_path = './Audio_Speech_Actors_01-24'
# X, binary_label = load_extract_features(data_path)
# print(X.shape)
# print(binary_label.shape)


In [8]:
# np.savetxt('matrix.txt', X, delimiter = ',')  
# np.savetxt('binary_label.txt',binary_label,delimiter = ',')

In [9]:
X = np.loadtxt(open("matrix.txt","rb"), delimiter=",", skiprows=0)
binary_label = np.loadtxt(open("binary_label.txt","rb"), delimiter=",", skiprows=0)

print(X.shape) #should be (768,140)
print(binary_label.shape) # should be (768,)

(768, 140)
(768,)


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, binary_label, test_size=0.3, random_state=0)

In [11]:
def compute_cost(W, X, Y, l):
    # calculate hinge loss
    n_s = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss =  (np.sum(distances) / n_s)
    # calculate cost
    cost = l / 2 * np.dot(W, W) + hinge_loss 
    
    return cost

In [12]:
def calculate_gradient(W, X, Y,l):

    distance = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if (ind == W.shape[0]):
            if max(0, d) != 0:
                di =  - ( Y[ind] * X[ind])
        else:
            if max(0, d) == 0:
                di = l*W
            else:
                di = l*W - ( Y[ind] * X[ind])
        dw += di

    dw = dw/len(Y)  # average

    return dw

In [13]:
def svm(X, Y, step, l,max_iter):
    X_bia = np.c_[X, np.ones(X.shape[0])]   # Pad 1's for the bias term
    new_Y = np.ones((Y.shape[0]))

    for i in range (Y.shape[0]):
        if Y[i] == 0:
            new_Y[i] = -1

    w = np.zeros(X_bia.shape[1])
    count = 0
    cost = compute_cost(w, X_bia, new_Y, l)
    for i in range(max_iter):
        grad = calculate_gradient(w, X_bia, new_Y, l)
        new_w = w - step * grad
        new_cost = compute_cost(new_w, X_bia, new_Y, l)
        print(cost)
        if (new_cost<cost):
            w = new_w
            cost = new_cost
            count= count +1
        else:
            break
    print(count)
    return w

In [14]:
def svm_accuracy(W,X,Y):

    X_bia = np.c_[X, np.ones(X.shape[0])]   # Pad 1's for the bias term
    new_Y = np.ones((Y.shape[0]))

    for i in range (Y.shape[0]):
        if Y[i] == 0:
            new_Y[i] = -1

    count = 0

    for i in range(X.shape[0]):
        yp = np.sign(np.dot(W, X_bia[i])) #model
        if(yp == new_Y[i]):
            count = count +1

    return count / Y.shape[0]

In [15]:
l = 13  # lambda
w_result = svm(X_train, Y_train, 0.00001, l,10000)
print(svm_accuracy(w_result, X_train, Y_train))
print(svm_accuracy(w_result, X_test, Y_test))

1.0
0.9998896765502112
0.9997793817559524
0.999669115609784
0.9995588781042685
0.99944866923197
0.9993384889854545
0.9992283373572912
0.9991182143400491
0.9990081199263009
0.9988980541086202
0.9987880168795829
0.9986780082317672
0.9985680281577523
0.9984580766501202
0.9983481537014547
0.9982382593043408
0.9981283934513664
0.9980185561351205
0.9979087473481946
0.9977989670831817
0.9976892153326773
0.9975794920892784
0.9974697973455832
0.997360131094194
0.9972504933277122
0.9971408840387436
0.9970313032198944
0.9969217508637732
0.9968122269629903
0.9967027315101588
0.9965932644978924
0.9964838259188072
0.9963744157655219
0.9962650340306564
0.9961556807068327
0.9960463557866748
0.9959370592628083
0.9958277911278609
0.9957185513744625
0.9956093399952446
0.9955001569828404
0.995391002329886
0.995281876029018
0.995172778072876
0.9950637084541009
0.9949546671653356
0.9948456541992258
0.9947366695484174
0.9946277132055596
0.9945187851633032
0.9944098854143008
0.9943010139512065
0.9941921707666

In [16]:
def pca(X, k):
    mean = np.mean(X, axis = 0)   # the mean value of X
    std = np.std(X, axis=0)       # the standard deviation along axis 0
    X_std = (X - mean) / std      # standardized data
    cov_mat = np.cov(X_std.T)     # calculate the covariance matrix

    values, vectors = np.linalg.eig(cov_mat)   # Eigendecomposition of covariance matrix

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(values[i]), vectors[i,:]) for i in range(len(values))]

    # Sort the tuples from the highest to the lowest based on eigenvalues magnitude
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Store the soted eigenvalues and their corresponding eigenvectors
    vecs_sorted = np.array([x[1] for x in eig_pairs])

    W = vecs_sorted[:k, :]   # Projection matrix with the top k eigenvectors selected

    return mean, W

In [17]:
mean, W = pca(X_train, 40)
print(mean.shape)

(140,)


In [18]:
X_proj = (X_train - mean).dot(W.T)             # transform X via W to obtain a k-dimensional feature subspace.
w = svm(X_proj, Y_train, 0.0001, l,2000)
print(svm_accuracy(w, X_proj, Y_train))

1.0
0.9994687874088684
0.9989389537892419
0.9984104955631258
0.9978834091618064
0.9973576910258276
0.9968333376049666
0.9963103453582112
0.9957887107537338
0.9952684302688688
0.9947495003900892
0.9942319176129821
0.993715678442225
0.9932007793915634
0.9926872169837858
0.9921749877507011
0.9916640882331146
0.9911545149808058
0.9906462645525037
0.9901393335158649
0.9896337184474492
0.9891294159326973
0.9886264225659083
0.9881247349502148
0.987624349697562
0.9871252634286837
0.9866274727730802
0.986130974368995
0.985635764863392
0.9851418409119337
0.984649199178958
0.984157836337456
0.9836677490690489
0.9831789340639665
0.9826913880210244
0.9822051076476018
0.9817200896596199
0.9812363307815174
0.9807538277462325
0.9802725772951768
0.9797925761782155
0.9793138211536453
0.9788363089881721
0.9783600364568888
0.977885000343255
0.9774111974390737
0.9769386245444701
0.9764672784678712
0.9759971560259828
0.9755282540437676
0.9750605693544269
0.9745940987993748
0.9741288392282205
0.9736647874987

In [19]:
X_proj = (X_test - mean).dot(W.T)       # transform X via W to obtain a k-dimensional feature subspace.
print(svm_accuracy(w, X_proj, Y_test))

0.6926406926406926
